In [16]:
import sys
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/database_functions/')
from database import Database
sys.path.append('H:/Tecnologia/EQUIPE - DADOS/dev_env/000_base/modules/')
from utils import Format_data
from datetime import datetime, timedelta
import numpy as np
import pandas as pd


db = Database()
fd = Format_data()

2025-10-15 13:39:12,283 - INFO - Cliente Oracle inicializado com sucesso usando o caminho: H:\Tecnologia\EQUIPE - DADOS\dev_env\000_base\instantclient_19_21
2025-10-15 13:39:12,353 - INFO - Banco de dados conectado com sucesso!


In [ ]:
# 🧠 Etapa 1: Consulta única de todas as campanhas -- 

# NOME TABELAS = 
tabelas_ = ['CAMP_CARDIOLOGIA', 'CAMP_CHECKUP_SAUDE', 'CAMP_CHECKUP_HOMEM', 'CAMP_CHECKUP_MULHER', 'CAMP_ODONTO', 
           'CAMP_OFTALMOLOGIA', 'CAMP_PSICO_PSIQUI','CAMP_ODONTO_GERAL', 'CAMP_DERMATOLOGIA', 'CAMP_ORTODONTIA']

# NOME CAMPANHAS = 
# campanhas = ['CARDIOLOGIA', 'CHECKUP MULHER', 'CHECK UP SAÚDE', 'OFTALMOLOGIA', 'PSICOLOGIA E PSIQUIATRIA', 'CHECKUP HOMEM',
#  'ODONTO', 'ODONTO GERAL','ORTODONTIA', 'DERMATOLOGIA','ORTODONTIA'
# ]


# NOME CAMPANHAS = 
campanhas = ['ODONTO GERAL']

c = db.extract_data(f"""
    SELECT * FROM CAMPANHAS_VOIP_BASE v
    WHERE v.NOME_CAMP IN ({','.join([f"'{camp}'" for camp in campanhas])})
""")


2025-10-15 13:39:28,452 - INFO - Dados coletados com sucesso!


In [ ]:
# 🧼 Etapa 2: Limpeza e filtragem dos dados

# 1. Limpa o telefone e calcula quantidade de dígitos
df_base_maling['QTDE_DIGITOS'] = df_base_maling['FONE_PRIMARIO_AJUST'].str.replace(r'[^0-9]', '', regex=True).str.len()

# 2. Filtra telefones válidos (mais de 10 dígitos e não nulos)
df_fones_validos = df_base_maling[
    (df_base_maling['QTDE_DIGITOS'] > 10) &
    (df_base_maling['FONE_PRIMARIO_AJUST'].notnull()) &
    (df_base_maling['FONE_PRIMARIO_AJUST'].str.contains(r'\d', na=False)) &
    (~df_base_maling['FONE_PRIMARIO_AJUST'].str.strip().str.startswith('0'))
].copy()

# 3. Remove duplicados por campanha, paciente e telefone
df_filtrado = df_fones_validos.drop_duplicates(
    subset=['NOME_CAMP', 'COD_PACIENTE'],
    keep='first'
).copy()

In [ ]:
# 📅 Etapa 3: Carregar calendário e preparar dias úteis

df_calendario = db.extract_data("""
    SELECT * FROM DADOS_CALENDARIO_BI dc
    WHERE dc.MES_AJUSTADO = '01/10/2025'
    AND dc.DATA > '01/10/2025'
""")

df_calendario['DATA'] = pd.to_datetime(df_calendario['DATA'], errors='coerce')
data_inicio = pd.to_datetime(datetime.today().date())

dias_uteis_validos = df_calendario[
    (df_calendario['FERIADO_FOLGA'] != 'S') &
    (df_calendario['DIAS_TT_MES'] > 0) &
    (df_calendario['DATA'] >= data_inicio)
][['DATA', 'DIAS_TT_MES']].copy()


2025-10-15 13:42:27,677 - INFO - Dados coletados com sucesso!


In [ ]:
# # 🧠 Etapa 3.5: Função para distribuir canais proporcionalmente
def distribuir_canais_por_campanha(df_filtrado, campanhas, total_canais= 25):
    contagem_por_campanha = {
        camp: len(df_filtrado[df_filtrado['NOME_CAMP'] == camp])
        for camp in campanhas
    }
    total_registros = sum(contagem_por_campanha.values())

    canais_por_campanha = {
        camp: max(1, int((qtde / total_registros) * total_canais))
        for camp, qtde in contagem_por_campanha.items()
    }

    # Ajuste fino: garantir que soma = total_canais
    sobra = total_canais - sum(canais_por_campanha.values())
    if sobra > 0:
        maiores = sorted(canais_por_campanha.items(), key=lambda x: -x[1])
        for i in range(sobra):
            canais_por_campanha[maiores[i % len(maiores)][0]] += 1

    return canais_por_campanha

# ✅ Executa distribuição de canais
canais_por_campanha = distribuir_canais_por_campanha(df_filtrado, campanhas, total_canais= 25)

In [29]:
# 🔁 Etapa 4: Loop por campanha com distribuição e insert

# Mapeamento seguro dos nomes reais das tabelas no banco
# tabelas_destino = {
#     'CARDIOLOGIA': 'AGE.CAMP_CARDIOLOGIA',
#     'CHECKUP MULHER': 'AGE.CAMP_CHECKUP_MULHER',
#     'CHECK UP SAÚDE': 'AGE.CAMP_CHECKUP_SAUDE',
#     'OFTALMOLOGIA': 'AGE.CAMP_OFTALMOLOGIA',
#     'PSICOLOGIA E PSIQUIATRIA': 'AGE.CAMP_PSICOLOGIA_PSIQUIATRIA',
#     'CHECKUP HOMEM': 'AGE.CAMP_CHECKUP_HOMEM',
#     'ODONTO': 'AGE.CAMP_ODONTO_GERAL',
#     'DERMATOLOGIA': 'AGE.CAMP_DERMATOLOGIA'
# }

tabelas_destino = {
    'ORTODONTIA': 'AGE.CAMP_ORTODONTIA'
}


for campanha in campanhas:
    df_campanha = df_filtrado[df_filtrado['NOME_CAMP'] == campanha].copy()
    total = len(df_campanha)
    if total == 0:
        continue

    # Expandir dias úteis proporcionalmente
    datas_expandidas = []
    for _, row in dias_uteis_validos.iterrows():
        peso = row['DIAS_TT_MES']
        rep = int(peso * 2)
        datas_expandidas.extend([row['DATA']] * rep)

    if len(datas_expandidas) == 0:
        raise ValueError(f"Nenhum dia útil disponível para campanha {campanha}")

    registros_por_dia = max(1, total // len(datas_expandidas))
    df_campanha['LIGAR_EM'] = [
        datas_expandidas[i // registros_por_dia] if i // registros_por_dia < len(datas_expandidas) else datas_expandidas[-1]
        for i in range(total)
    ]
    df_campanha['LIGAR_EM'] = pd.to_datetime(df_campanha['LIGAR_EM']).dt.strftime('%Y-%m-%d')

    # 🔄 Distribuir canais proporcionalmente dentro da campanha
    qtde_canais = 3 # canais_por_campanha[campanha]
    canais_disponiveis = list(range(1, qtde_canais + 1))
    df_campanha['CANAIS'] = [canais_disponiveis[i % qtde_canais] for i in range(total)]

    # 🧼 Inserir no banco
    tabela_destino = tabelas_destino.get(campanha)
    if not tabela_destino:
        raise ValueError(f"Tabela não mapeada para campanha: {campanha}")

    for _, row in df_campanha.iterrows():
        sql = f"""
            INSERT INTO {tabela_destino} (
                SEQ_CAMPANHA,
                COD_PACIENTE,
                NOME_PACIENTE,
                FONE,
                CAMPANHA,
                STATUS,
                LIGAR_EM,
                CANAIS
            ) VALUES (
                SEQ_CAMPANHA.nextVal,
                :1, :2, :3, :4, :5, TO_DATE(:6, 'YYYY-MM-DD'), :7
            )
        """
        db.cursor.execute(sql, (
            row['COD_PACIENTE'],
            str(row['NOME']).replace("'", "''"),
            str(row['FONE_PRIMARIO_AJUST']).replace("'", "''"),
            str(row['NOME_CAMP']).replace("'", "''"),
            'Enviar',
            row['LIGAR_EM'],
            row['CANAIS']
        ))


In [30]:
# Commit final
db.connection.commit()

In [ ]:
# Exclui fones duplicados nas tabeas induviduais se tiverem.

for tabela in campanhas:
    print(f"\n🔍 Validando campanha: {tabela}")

df_base_limpa = f""" 
DELETE FROM {tabela}
WHERE ROWID IN (
    SELECT rid FROM (
        SELECT 
            ROWID AS rid,
            ROW_NUMBER() OVER (
                PARTITION BY CAMPANHA, COD_PACIENTE, REGEXP_REPLACE(FONE, '[^0-9]', '')
                ORDER BY LIGAR_EM ASC
            ) AS rn
        FROM CAMP_ODONTO_GERAL
        WHERE TO_CHAR(LIGAR_EM, 'MM/YYYY') = '10/2025'
    )
    WHERE rn > 1
)
 """